# Аналитическая модель определения параметров выходного потока, имеющего равномерное распределение, по значениям параметров входного потока, имеющим распределение Вейбулла

In [1]:
import numpy as np
from sympy import *
from IPython.display import display, Math

Инициализировать средства отображения данных SymPy.

In [2]:
init_printing()

Задать переменные, задействованные в математических выражениях.

In [3]:
# Параметры распределения Вейбулла
a = Symbol(r'\alpha', positive=True)
s = Symbol('s', positive=True)

# Параметры равномерного распределения
l = Symbol('l', positive=True)
h = Symbol('h', positive=True)

# Сводобная переменная
t = Symbol(r'\tau', real=True)

Определить описание распределения Вейбулла:

In [4]:
fp = (a / s) * (t / s) ** (a - 1) * exp(-(t / s) ** a)
f = Piecewise(
    (fp, t >= 0),
    (0.0, True)
)

display(Math(fr"f(\tau) = {latex(f)}"))  # noqa

<IPython.core.display.Math object>

Найдем величину энтропии для входного потока, имеющего распределение
Вейбулла.

In [5]:
Hf = -Integral(fp * log(fp), (t, 0, oo))
display(Math(f"H(f) = {latex(Hf)}"))  # noqa

<IPython.core.display.Math object>

Определить описание равномерного распределения:

In [6]:
gp = 1 / (h - l)
g = Piecewise(
    (gp, And(t >= l, t <= h)),
    (0.0, True)
)
display(Math(fr"g(\tau) = {latex(g)}"))  # noqa

<IPython.core.display.Math object>

Найдем величину кроссэнтропии между потоком, имеющим распределение Парето и
потоком, имеющим экспоненциальное распределение

In [7]:
Hfg = -log(gp) * integrate(fp, (t, l, h))
display(Math(f"H(f, g) = {latex(Hfg)}"))  # noqa

<IPython.core.display.Math object>

Упростив это выражением, получим

In [8]:
Hfg = Hfg.subs(log(1 / (h - l)), -log(h - l))
display(Math(f"H(f, g) = {latex(Hfg)}"))  # noqa

<IPython.core.display.Math object>

Зная значения энтропии $H(f)$ и кроссэнтропии $H(f, g)$ найдем величину дивергенции
Кульбака-Лейблера

In [9]:
Dkl = Hfg - Hf
display(Math(f"D_{{KL}} = {latex(Dkl)}"))  # noqa

<IPython.core.display.Math object>

Найдем производную дивергенции Кульбака-Лейблера по параметрам выходного
потока $l$ и $h$:

In [10]:
dDkll = diff(Dkl, l)
display(Math(fr"\frac{{\partial\,D_{{KL}}}}{{\partial\,l}} = {latex(dDkll)}"))  # noqa

dDklh = diff(Dkl, h)
display(Math(fr"\frac{{\partial\,D_{{KL}}}}{{\partial\,h}} = {latex(dDklh)}"))  # noqa

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Приравняв к нуля значение производных, и решив полученное уровнение
относительно $l$ и $h$ найдем

In [11]:
solution = nonlinsolve((dDkll, dDklh), (l, h))  # noqa
display(Math(latex(solution)))  # noqa

<IPython.core.display.Math object>

Решение этой системы уравнений не может быть получено аналитически. Найдем
численное решение этой системы уравнений для коэффициента масштаба $s = 1$.

In [12]:
dDkll1 = dDkll.subs(s, 1)
display(Math(fr"\frac{{\partial\,D_{{KL}}}}{{\partial\,l}} = {latex(dDkll1)}"))  # noqa

dDklh1 = dDklh.subs(s, 1)
display(Math(fr"\frac{{\partial\,D_{{KL}}}}{{\partial\,h}} = {latex(dDklh1)}"))  # noqa

<IPython.core.display.Math object>

<IPython.core.display.Math object>

спользуя соотношение $\alpha = 2 - 2H$, найдем решение этой системы уравнений
при различных значениях показателя Херста $H$.

In [13]:
for H in np.arange(0.5, 1.0, 0.05):
    alpha = 2.0 - 2.0 * H

    dDkll1a = dDkll1.subs(a, alpha)
    dDklh1a = dDklh1.subs(a, alpha)

    try:
        solution = nsolve((dDkll1a, dDklh1a), (l, h), (0.01, 100.0))
        print(solution)
    except ValueError:
        print(f"Нет решения для H = {H}")
    except ZeroDivisionError:
        print(f"Нет решения для H = {H} (вырожденная матрица)")

Нет решения для H = 0.5
Нет решения для H = 0.55
Нет решения для H = 0.6000000000000001
Нет решения для H = 0.6500000000000001
Нет решения для H = 0.7000000000000002
Нет решения для H = 0.7500000000000002
Нет решения для H = 0.8000000000000003
Нет решения для H = 0.8500000000000003
Нет решения для H = 0.9000000000000004
Нет решения для H = 0.9500000000000004


Таким образом, при преобразовании распределения Вейбулла в равномерное
распределение отсутствует оптимальный списоб получения границ интервала
$l$ и $h$ равномерного распределения по коэффициентам формы $\alpha$
и масштаба $s$ распределения Вейбулла, при которых величина дивергенции
Кульбака-Лейблера будет минимальной.